In [ ]:
########## 벤치마크 데이터 만들기 (빅카인즈 기사 6만건) ##########

##### API로 데이터 불러오기 #####

from urllib import request, parse
import pandas as pd
import json
from pandas.io.json import json_normalize

headers = {'Content-Type': 'application/json; chearset=utf-8'}

# news_provider = ['경향신문','국민일보','내일신문','동아일보', '문화일보','서울신문','세계일보','조선일보', '중앙일보','한겨례','한국일보','경기일보', '경인일보','강원도민일보','강원일보','대전일보',
#                 '중도일보','중부매일','중부일보','충북일보', '충청일보','충청투데이','경남신문','경남도민일보', '경상일보','국제신문','대구일보','매일신문', '부산일보','영남일보','울산매일','광주매일신문',
#                 '광주일보','무등일보','전남일보','전북도민일보', '전북일보','제민일보','한라일보','매일경제','머니투데이', '서울경제','파이낸셜뉴스','한국경제','헤럴드경제','아시아경제', '아주경제','전자신문','디지털타임스']
# no_society_provider = ['한겨례', '한국일보', '매일경제'] : 세개 신문사에는 '사회'카테고리에 보도된 기사 없음

major_provider = ['조선일보', '중앙일보', '동아일보']

json_file1 = pd.DataFrame()

for i in major_provider :
  data = {
      'access_key': 'c623a12a-2a15-44bc-8d93-183b5b6834b4',
      'argument': {
          #'query' : '취업',
          'published_at': {
              'from' : '2021-08-01',
              'until' : '2022-08-01'
              },
          'provider' : [i],
          'category' : ['사회', '경제', '정치', '문화'],
          'return_from' : 1,
          'return_size' :10000,
          'fields' : ['content' ,'category']
        }
        }
  req = request.Request('http://tools.kinds.or.kr:8888/search/news', headers=headers, data=json.dumps(data).encode('utf-8'))
  res = request.urlopen(req)

  json_str = res.read().decode('utf-8')
  json_object = json.loads(json_str)
  json_df = json_normalize(json_object['return_object']['documents'])
  json_file1 = pd.concat([json_file1,json_df], ignore_index=True)

json_file2 = pd.DataFrame()

for i in major_provider :
  data = {
      'access_key': 'c623a12a-2a15-44bc-8d93-183b5b6834b4',
      'argument': {
          #'query' : '취업',
          'published_at': {
              'from' : '2021-08-01',
              'until' : '2022-08-01'
              },
          'provider' : [i],
          'category' : ['사회', '경제', '정치', '문화'],
          'return_from' : 10001,
          'return_size' :10000,
          'fields' : ['content' ,'category']
        }
        }
  req = request.Request('http://tools.kinds.or.kr:8888/search/news', headers=headers, data=json.dumps(data).encode('utf-8'))
  res = request.urlopen(req)

  json_str = res.read().decode('utf-8')
  json_object = json.loads(json_str)
  json_df = json_normalize(json_object['return_object']['documents'])
  json_file2 = pd.concat([json_file2,json_df], ignore_index=True)


# 예비용
json_file3 = pd.DataFrame()

for i in major_provider :
  data = {
      'access_key': 'c623a12a-2a15-44bc-8d93-183b5b6834b4',
      'argument': {
          #'query' : '취업',
          'published_at': {
              'from' : '2021-08-01',
              'until' : '2022-08-01'
              },
          'provider' : [i],
          'category' : ['사회', '경제', '정치', '문화'],
          'return_from' : 20001,
          'return_size' :1000,
          'fields' : ['content' ,'category']
        }
        }
  req = request.Request('http://tools.kinds.or.kr:8888/search/news', headers=headers, data=json.dumps(data).encode('utf-8'))
  res = request.urlopen(req)

  json_str = res.read().decode('utf-8')
  json_object = json.loads(json_str)
  json_df = json_normalize(json_object['return_object']['documents'])
  json_file3 = pd.concat([json_file3,json_df], ignore_index=True)

In [ ]:
##### rawraw : API로 불러온 63000개의 기사 #####
rawraw = pd.concat([json_file1, json_file2, json_file3], ignore_index=True)

##### benchmark : 작업 진행 할 데이터 프레임 #####
benchmark = pd.DataFrame()
benchmark = pd.concat([benchmark, rawraw], ignore_index= True)

In [ ]:
##### 데이터 전처리 #####
##### title [부음], [인사]로 시작하는 기사 제거
for i in range(len(benchmark)) :
  if benchmark['title'][i].startswith('[부음]') or benchmark['title'][i].startswith('[인사]') :
    benchmark.drop(i, inplace=True)
len(benchmark)

In [ ]:
##### 중복기사제거
remove_index = benchmark[benchmark.duplicated(['content'])].index
benchmark.drop(remove_index, inplace=True)
len(benchmark)

In [ ]:
###### benchmark 데이터 6만개로 맞추기 #####
benchmark = benchmark[:60000]
benchmark.reset_index(drop = True, inplace = True)  # 인덱스 초기화

In [ ]:
##### csv 파일로 저장 #####
# benchmark.to_csv('benchmark_data_60000.csv', encoding = 'utf-8-sig')

In [ ]:
########## 데이터 전처리 ##########
import re

In [ ]:
###### 본문만 사용할 거니까 content 시리즈로 만들기 #####
news_content = benchmark['content']
news_content

In [ ]:
## 전처리 완료된 content 데이터 : newsnews
newsnews = news_content
newsnews = newsnews.str.replace("\(.*\)|\s-\s.*"," " ,regex=True)
newsnews = newsnews.str.replace("\[.*\]|\s-\s.*"," ",regex=True)
newsnews = newsnews.str.replace("\<.*\>|\s-\s.*"," ",regex=True)
newsnews = newsnews.str.replace("무단전재 및 재배포 금지"," ",regex=True)
newsnews = newsnews.str.replace("무단 전재 및 재배포 금지"," ",regex=True)
newsnews = newsnews.str.replace("©"," ",regex=True)
newsnews = newsnews.str.replace("ⓒ"," ",regex=True)
newsnews = newsnews.str.replace("저작권자"," ",regex=True)
newsnews = newsnews.str.replace(".* 기자", " ", regex=True) #기자 이름에서 오는 유사도 차단
newsnews = newsnews.str.replace("사진 = .*", " ", regex=True) #사진 첨부 문구 삭제
newsnews = newsnews.str.replace("사진=.*", " ", regex=True) #사진 첨부 문구 삭제
newsnews = newsnews.str.replace('\"', "",regex=True)
newsnews = newsnews.str.replace("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+.[a-zA-Z0-9-.]+)", " ", regex=True) #이메일 주소에서 오는 유사도 차단
newsnews = newsnews.str.replace("\n"," ")
newsnews = newsnews.str.replace("\r"," ")
newsnews = newsnews.str.replace("\t"," ")
newsnews = newsnews.str.replace( "\’" , "", regex=True)
# newsnews = newsnews.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
newsnews = newsnews.str.replace("[ ]{2,}"," ",regex=True)

In [ ]:
########## 문장토큰화 ##########
!pip install kss
import kss
import numpy as np

In [ ]:
con_sen_li = []
for i in range(len(newsnews)) :
  con_sen_li.append(kss.split_sentences(newsnews[i]))

In [ ]:
##### final_data : 뽑은 피처값들과 필요한 데이터들을 하나의 데이터프레임으로 만들기 #####
keep_columns = ['news_id', 'title', 'content', 'enveloped_at', 'provider']
final_data = benchmark[keep_columns]

In [ ]:
##### 토큰화 된 문장 final_data에 새로운 column으로 붙이기 #####
final_data = final_data.assign(con_sen_li = con_sen_li)

In [ ]:
##### 토큰화 된 문장 수 구해서 final_data에 새로운 column으로 붙이기 #####
count_sen = [0] *60000
for i in range(len(con_sen_li)) :
  count_sen[i] = len(con_sen_li[i])

final_data = final_data.assign(count_sen = count_sen)

In [ ]:
final_data.to_csv('final_final_data.csv', encoding = 'utf-8-sig') #(데이터 손실 방지하기 위해 틈틈이 CSV로 저장)

In [ ]:
########## pos tagging : numpy 배열로 하면 시간 절약 ##########
content_array = newsnews.to_numpy()

In [ ]:
pip install konlpy

In [ ]:
from konlpy.tag import Okt

In [ ]:
con_pos = []
for i in range(len(content_array)) :
  okt = Okt()
  con_pos.append(okt.pos(content_array[i]))

In [ ]:
##### pos 태깅 된 데이터 final_data에 새로운 column으로 붙이기 #####
final_data = final_data.assign(con_sen_li = con_sen_li)
final_data = final_data.assign(con_pos = con_pos)

In [ ]:
##### pos 태깅 된 데이터에서 기사별 품사 개수 새로운 column으로 붙이기 #####
pos = []
count_noun = []
count_adjective = []
count_verb = []
count_determiner = []
count_adverb = []
count_conjunction = []
count_josa = []

for con_index in range(len(con_pos)) :
  pospos = []
  for sen_index in range(len(con_pos[con_index])) :
    pospos.append(con_pos[con_index][sen_index][1])
  pos.append(pospos)
  count_noun.append(pos[con_index].count('Noun'))
  count_adjective.append(pos[con_index].count('Adjective'))
  count_verb.append(pos[con_index].count('Verb'))
  count_determiner.append(pos[con_index].count('Determiner'))
  count_adverb.append(pos[con_index].count('Adverb'))
  count_conjunction.append(pos[con_index].count('Conjunction'))
  count_josa.append(pos[con_index].count('Josa'))

In [ ]:
##### 기사별 품사 개수 final_data에 새로운 column으로 붙이기 #####
final_data = final_data.assign(count_noun = count_noun)
final_data = final_data.assign(count_adjective = count_adjective)
final_data = final_data.assign(count_verb = count_verb)
final_data = final_data.assign(count_determiner = count_determiner)
final_data = final_data.assign(count_adverb = count_adverb)
final_data = final_data.assign(count_conjunction = count_conjunction)
final_data = final_data.assign(count_josa = count_josa)

In [ ]:
final_data.to_csv('final_final_data.csv', encoding = 'utf-8-sig') #(데이터 손실 방지하기 위해 틈틈이 CSV로 저장)